## <a name="abstract">Titanic Machine Learning From Disaster</a>

Abstract The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.
One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.
In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

## Import External Notebook: Julia Assistant Tools

In [146]:
using NBInclude
using DecisionTree
# nbinclude("../06_assistent_tools/Julia\ Assistent\ Tools\ for\ Machine\ Learning.ipynb")

## Load Data

In [73]:
train = readtable("data/train.csv")
test = readtable("data/test.csv")
full = vcat(train, test);

## Define Features X and Label Y

In [76]:
features = names(test)
label = setdiff(names(train), names(test));

## Remove Missing Data

### Get Default Values

In [78]:
default_values = get_default_values(full, features)

Dict{Any,Any} with 11 entries:
  :Parch       => 0
  :PassengerId => 1
  :Age         => 24.0
  :Pclass      => 3
  :Name        => "Kelly, Mr. James"
  :Ticket      => "CA. 2343"
  :Cabin       => "C23 C25 C27"
  :Sex         => "male"
  :Embarked    => "S"
  :SibSp       => 0
  :Fare        => 8.05

### Apply Default Values

In [80]:
apply_default_values!(train, default_values)
apply_default_values!(test, default_values)
apply_default_values!(full, default_values);

## Add New Features

### Feature 1: Title

In [81]:
title_sir = ["Capt", "Don", "Major", "Master", "Sir", "Dr"]
title_lady = ["Don", "Lady", "the Countess", "Jonkheer"]
training_title_list = map(n -> split(strip(split(n, ",")[2]), ".")[1], train[:Name])
test_title_list = map(n -> split(strip(split(n, ",")[2]), ".")[1], test[:Name]);

In [82]:
is_in_title_list(names, title_list) = Array{Bool}(map(n -> n in title_list, names))
train[:Title] = training_title_list
train[is_in_title_list(training_title_list, title_sir), :Title] = "Sir."
train[is_in_title_list(training_title_list, title_lady), :Title] = "Lady."
test[:Title] = test_title_list
test[is_in_title_list(test_title_list, title_sir), :Title] = "Sir."
test[is_in_title_list(test_title_list, title_lady), :Title] = "Lady.";

### Feature 2: Family Size

In [83]:
train[:FamilySize] = train[:Parch] .+ train[:SibSp] .+ 1
test[:FamilySize] = test[:Parch] .+ test[:SibSp] .+ 1;

### Feature 3: Is Mother

In [84]:
is_mother(df) = Array{Bool}(map(i -> df[i, :Parch] > 0 && df[i, :Age] > 18 && df[i, :Title] != "Miss", vcat(1:size(df,1))))
train[:IsMother] = 0
train[is_mother(train), :IsMother] = 1
test[:IsMother] = 0
test[is_mother(test), :IsMother] = 1;

### Feature 4: Is Child

In [85]:
is_child(df) = Array{Bool}(map(i -> df[i, :Parch] > 0 && df[i, :Age] <= 18, size(df,1)))
train[:IsChild] = 0
train[is_child(train), :IsChild] = 1
test[:IsChild] = 0
test[is_child(test), :IsChild] = 1;

### Feature 5: Family Size (2)

In [86]:
get_surname(name) = strip(split(name, ",")[1])
surname_freq = countmap(map(n -> get_surname(n), vcat(train[:Name], test[:Name])))
is_small_family(names) = map(n -> surname_freq[get_surname(n)] > 2? 0 : 1, names);
train[:IsSmallFamily] = is_small_family(train[:Name])
test[:IsSmallFamily] = is_small_family(test[:Name]);

### Feature 6: Cabin Prefix and Number

In [98]:
get_cabin_prefix(cabins) = map(c -> !isna(c)? string(c[1]) : NA, cabins)
get_cabin_number(cabins) = Array{AbstractString}(map(c ->  !isna(c)? string(c[2:end]) : "-1", cabins))
train[:CabinPrefix] = get_cabin_prefix(train[:Cabin])
train[:CabinNumber] = get_cabin_number(train[:Cabin])
test[:CabinPrefix] = get_cabin_prefix(test[:Cabin])
test[:CabinNumber] = get_cabin_number(test[:Cabin]);

## Encode Categorical Features

In [106]:
categorical_features = [:Pclass, :Sex, :Ticket, :Cabin, :Embarked, :Title, :CabinPrefix, :CabinNumber]
full = vcat(train, test);

In [110]:
encoding = get_label_encoding(full, categorical_features)
apply_encoding!(train, encoding)
apply_encoding!(test, encoding)
apply_encoding!(full, encoding);

## Apply One-Hot-Encoding

In [116]:
all_values = get_all_values(full, categorical_features)
apply_one_hot_encoding!(train, categorical_features, all_values)
apply_one_hot_encoding!(test, categorical_features, all_values)
apply_one_hot_encoding!(full, categorical_features, all_values);

	Processed Categorical Feature 1000/1309.


## Apply Interval Features

In [148]:
interval_features = [:Age, :Fare, :FamilySize]
intervals = Dict(:Fare => [0, 1, 5, 10, 25, 50, 100], 
                 :FamilySize => [0, 1, 2, 3, 5],
                 :Age => [14, 18, 21, 30, 40, 50, 60])
apply_interval_features!(train, interval_features, intervals)
apply_interval_features!(test, interval_features, intervals);

## Export New Train/Test Data

In [149]:
writetable("data/train_v2.tsv", train, separator='\t')
writetable("data/test_v2.tsv", test, separator='\t');